In [20]:
import chess

import ipywidgets as widgets
import chess.pgn
import numpy as np
import io
pgn='''[Event "Biel Chess Festival"]
[Site "Biel SUI"]
[Date "2007.08.02"]
[Round "9"]
[White "Magnus Carlsen"]
[Black "Teimour Radjabov"]
[Result "1-0"]
[EventDate "?"]
[ECO "B07"]
[WhiteElo "2710"]
[BlackElo "2747"]
[PlyCount "55"]

1. e4 d6 2. d4 Nf6 3. Nc3 e5 4. Nge2 Nbd7 5. g3 c6 6. Bg2 b5 7. a3 Be7 8. O-O
O-O 9. h3 a5 10. g4 Ba6 11. Ng3 b4 12. Nce2 bxa3 13. Rxa3 d5 14. Re3 dxe4 15.
Re1 Qc7 16. Nf5 Bd8 17. g5 Nd5 18. Rxe4 f6 19. Neg3 g6 20. Nh6+ Kg7 21. dxe5
fxg5 22. e6 Kxh6 23. e7 Qb6 24. exf8=Q+ Nxf8 25. c4 Nf4 26. Qd6 Kg7 27. Bxf4
gxf4 28. Re7+ 1-0'''

In [80]:
SIGMA= 5
FRACFACTOR=2
SCORECUTOFF=200
STOCKFISHPATH=r'c:/gitproj/stockfish/stockfish-windows-x86-64-avx2.exe'
STOCKFISHDEPTH=10
import nest_asyncio
nest_asyncio.apply()
from ipywidgets import interact, interactive, fixed, interact_manual, SelectMultiple, Combobox, HBox, VBox
import ipywidgets as widgets
from collections import defaultdict
import asyncio
#b = Button(description='HideShow')
import chess.engine
from chess import Board 
def get_score(b,white):
    cc= engine.analyse(b, chess.engine.Limit(depth=STOCKFISHDEPTH))['score']
    
    if type(cc.relative) is chess.engine.Mate or  type(cc.relative) is chess.engine.MateGiven:
        if cc.relative< chess.engine.Cp(-500):
            return -5000
        else:
            return 5000
    
    
    if cc.relative.score() is None:
        print(cc)
        print(cc.score())
        breakpoint()
    return cc.relative.score() * (1 if white else (-1))
    
#Box( current_board
from stockfish import Stockfish
engine = chess.engine.SimpleEngine.popen_uci(STOCKFISHPATH)



async def calc_moves_score_int(cur, white, curdep, maxdepth, oldeval, l):
    from aiostream import stream
    fen=cur.fen()
    from aiostream import stream, pipe

    async with stream.iterate(cur.generate_legal_moves()).stream() as it:
        async for z in it: #stream.list(cur.generate_legal_moves()):
            cur=Board(fen=fen)
            cur.push(z)
            ev = get_score(cur, not white)
            g = ev - oldeval
            if white and g < (-1) * SCORECUTOFF: #too bad
                cur.pop()
                continue 
            elif g > SCORECUTOFF:
                cur.pop()
                continue
            
            l[curdep] += 1
            
            if curdep == maxdepth:
                yield ev
            else:
                async for k in  calc_moves_score_int(cur, not white, curdep + 1, maxdepth, ev, l):
                    yield k
            
            cur.pop()
def print_stats(curb,iswhite):
    try:
      
        init,origg,stab,b,lev =  calc_stability(curb,iswhite)
        ls=['score','stability factor' , 'num of reasonable moves' , 'max(score) of reasonable','min(score)  of reasonable','faraction method','moves by depth']
        tup=('%.2f' % (init/100),('%.2f' % (stab*100))+'%',len(origg),max(origg),min(origg),b,dict(lev))
        for a,b in zip(ls,tup):
            print(a+ ': '+str(b))
    except Exception as e:
        import traceback
        print(traceback.format_exc())

def calc_moves_score(cur,white,oldeval,depth=2):
    from aiostream import stream

    async def tmp():
        xx=  await stream.list(calc_moves_score_int(cur, white, 1, depth, oldeval, lev))
        return xx
        
    lev= defaultdict(int)
    ll = asyncio.run(tmp())
    return ll,lev

def calc_stability(cur_board,iswhite):
    '''
    calcs stability by first getting score of all reasonable moves, then apply calculation(see readme). 
    returns initial score, vector of diff vs initial score, stability factor, if fraction method
    '''
    init=get_score(cur_board,iswhite)
    arr,lev=calc_moves_score(cur_board,iswhite,init)
    g=np.array(arr,dtype="float64")
    g-=init
    g/=100
    
    orig_vec=np.copy(g)
    b=False
    if abs(init)>200:
        b=True
        g=g/(init/100) * FRACFACTOR #TODO:to make continous...
    else:
        b=False
    
    g= (-1)*np.square(g) *SIGMA
    g=np.exp(g)
    stab=np.sum(g)/len(g)
    return init,orig_vec,stab,b,lev
    
def display_game(pgn):      
    def rev(x):
        a.value=(a.value-1)
    def adv(x):
        a.value=(a.value+1)
    p=chess.pgn.read_game(io.StringIO(pgn))
    gam=p.game()
    current_board= widgets.HTML( str(gam.board()._repr_svg_())) 
    from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider
    
    butl=Layout(width='35px',justify_content='center')
    ba=widgets.Button(description='<',layout=butl)
    ba.on_click(rev)
    ne=widgets.Button(description='>',layout=butl)
    ne.on_click(adv)
    mmax=len(list(gam.mainline_moves()))
    a = widgets.IntSlider(value=0, min=0, max=mmax)
    items = [ba,ne, widgets.Button(description='R',layout=butl),a]
    

    gb2 = widgets.HBox(items,layout=Layout(display='flex',
            flex_flow='row',
            align_items='stretch',
            width='100%'))
    def f2(a):
        cur=gam
        for k in range(a):
            cur=cur.next()
        display( widgets.HTML(str(cur.board()._repr_svg_())))
    
                
    def f3(a):
        cur=gam
        for k in range(a):
            cur=cur.next()
        
        iswhite= (a%2 ==0)
        
        curb=cur.board()
        #display(list(curb.generate_legal_moves()))
        print_stats(curb,iswhite)
            
        
        
        
    
    return VBox([gb2,widgets.interactive_output(f2,{'a':a}),widgets.interactive_output(f3,{'a':a})],layout=Layout(
        display='flex',
        flex_flow='column',
        align_items='stretch',
        width='90%'
    ) )

def display_board(fen,iswhite=None):
    b=chess.Board(fen)
    if iswhite is None:
        iswhite= b.turn
    display( widgets.HTML(str(b._repr_svg_())))
    print_stats(b,iswhite)
    
    
p=chess.pgn.read_game(io.StringIO(pgn))
gg=p.game()
#list(calc_b(gg.next().next().next(),False,-55))


In [68]:
display_game(pgn)

In [ ]:


display_game(pgn)

## my stupid game

In [23]:
pgn2='''
[Event "Live Chess"]
[Site "Chess.com"]
[Date "2023.12.05"]
[Round "?"]
[White "nevertheless33"]
[Black "andrewmoussa"]
[Result "1-0"]
[ECO "D01"]
[TimeControl "300"]
[EndTime "14:11:45 PST"]
[Termination "nevertheless33 won by resignation"]

1. d4 d5 2. Nc3 Nf6 3. Bg5 Ne4 4. Nxe4 dxe4 5. Qd2 h6 6. Be3 Bf5 7. h3 e6 8. g4
Bg6 9. O-O-O Be7 10. Kb1 O-O 11. Bf4 Nd7 12. Qc3 c6 13. e3 Bf6 14. Ne2 Re8 15.
Ng3 e5 16. dxe5 Nxe5 17. Rxd8 Raxd8 18. Be2 Nc4 19. Qxc4 b5 20. Qxc6 Re6 21. Qc7
Ra6 22. Rd1 Rxd1+ 23. Bxd1 Rb6 24. Bd6 Ra6 25. Qc8+ 1-0'''

In [81]:
display_game(pgn2)

In [79]:
display_board('r2Rr1k1/pp3pp1/2p2bbp/4n3/4pBP1/2Q1P1NP/PPP2P2/1K3B1R b - - 0 17',True)

HTML(value='<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" viewBox="0 0 39…

score: -3.72
stability factor: 38.85%
num of reasonable moves: 14
max(score) of reasonable: 2.25
min(score)  of reasonable: -0.12
faraction method: True
moves by depth: {1: 2, 2: 14}
